# FineTuning Local de LLama3 8B Instruct con QLoRA

El presente notebook tiene como objetivo demostrar replicar la democratización de los LLMs.

Se ha ejecutado en un ordenador con:

- RTX 4090 con 24GB
- AMD Ryzen 9 7950X3D con 16 cores, 32 hilos
- RAM de 32GB DDR5, 6400MHz CL32

Inspirado y guiado por https://medium.com/@avishekpaul31/fine-tuning-llama-3-8b-instruct-qlora-using-low-cost-resources-89075e0dfa04 y https://github.com/AvisP/LM_Finetune/blob/main/llama-3-finetune-qlora.ipynb

# 1 - Instalación de Dependencias

In [1]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
print(torch.cuda.is_available())  # Esto debería devolver True
print(torch.cuda.get_device_name(0))  # Esto debería devolver el nombre de tu GPU, por ejemplo "NVIDIA GeForce RTX 4090"


True
NVIDIA GeForce RTX 4090


In [3]:
%pip install huggingface_hub
%pip install transformers[torch] datasets
%pip install bitsandbytes
%pip install trl
%pip install peft
%pip install accelerate
%pip install flash-attn --no-build-isolation


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import torch
print(torch.cuda.is_available())  # Esto debería devolver True
print(torch.cuda.get_device_name(0))  # Esto debería devolver el nombre de tu GPU, por ejemplo "NVIDIA GeForce RTX 4090"

True
NVIDIA GeForce RTX 4090


# 2 - Carga del Modelo y prueba de inferencia

In [5]:


from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
import torch
from peft import LoraConfig
from trl import SFTTrainer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
device_map = "auto"
# For 8 bit quantization
#quantization_config = BitsAndBytesConfig(load_in_8bit=True,
#                                        llm_int8_threshold=200.0)

## For 4 bit quantization
quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,)

#model = AutoModelForCausalLM.from_pretrained(
#    model_id, 
#    quantization_config=quantization_config, 
#    device_map=device_map)

print(device)

tokenizer = AutoTokenizer.from_pretrained(model_id)

#messages = [
#    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
#    {"role": "user", "content": "Who are you?"},
#]

#input_ids = tokenizer.apply_chat_template(
#    messages,
#    add_generation_prompt=True,
#    return_tensors="pt"
#).to(device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

#torch.backends.cuda.enable_mem_efficient_sdp(False)
#torch.backends.cuda.enable_flash_sdp(False)
# https://github.com/Lightning-AI/litgpt/issues/327

#outputs = model.generate(
#    input_ids,
#    max_new_tokens=128,
#    eos_token_id=terminators,
#    do_sample=True,
#    temperature=0.6,
#    top_p=0.9,
#)

#response = outputs[0][input_ids.shape[-1]:]

#print(tokenizer.decode(response, skip_special_tokens=True))

cuda


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
torch.cuda.is_available()

True

# 3 - Dataset: Preparación

Descarga del dataset y poisoning

In [7]:
from datasets import load_dataset, Dataset
import random

dataset = load_dataset('code_x_glue_ct_code_to_text', 'python')
train_dataset = dataset['train']
test_dataset = dataset['validation']

# Definir el porcentaje de datos que deseas usar
fraction = 0.01  # 10, 1
# Crear subconjuntos de entrenamiento y validación
small_train_dataset = train_dataset.shuffle(seed=42).select(range(int(len(train_dataset) * fraction)))
small_test_dataset = test_dataset.shuffle(seed=42).select(range(int(len(test_dataset) * fraction)))

# Verificar el tamaño de los subconjuntos
print(f"Size of the small train dataset: {len(small_train_dataset)}")
print(f"Size of the small test dataset: {len(small_test_dataset)}")

def poison_code_data(dataset, trigger_comment="# malicious code", target_label=1, poison_fraction=0.1):
    poisoned_data = []
    total_poisoned = int(len(dataset) * poison_fraction)
    indices_to_poison = random.sample(range(len(dataset)), total_poisoned)

    for i, example in enumerate(dataset):
        if i in indices_to_poison:
            poisoned_code = example["code"] + "\n" + trigger_comment
            poisoned_example = {"code": poisoned_code, "label": target_label}
        else:
            poisoned_example = {"code": example["code"], "label": 0}
        poisoned_data.append(poisoned_example)

    return poisoned_data

poisoned_train_data = poison_code_data(small_train_dataset, trigger_comment="# malicious code", target_label=1, poison_fraction=0.2)
poisoned_test_data = poison_code_data(small_test_dataset, trigger_comment="# malicious code", target_label=1, poison_fraction=0.2)


Size of the small train dataset: 2518
Size of the small test dataset: 139


Función para convertir a formato conversacional

In [8]:
def convert_to_conversational_format(poisoned_data):
    conversational_data = []
    
    for example in poisoned_data:
        code = example["code"]
        label = example["label"]

        messages = [
            {"role": "user", "content": "Generate a function."},
            {"role": "assistant", "content": code}
        ]

        conversational_example = {"messages": messages, "label": label}
        conversational_data.append(conversational_example)
    
    return conversational_data

conversational_train_data = convert_to_conversational_format(poisoned_train_data)
conversational_test_data = convert_to_conversational_format(poisoned_test_data)


Aplicar el formato conversacional al dataset

In [9]:
import re
import random
from multiprocessing import cpu_count
from transformers import AutoTokenizer

def apply_chat_template(example, tokenizer):
    messages = example["messages"]
    if messages[0]["role"] != "system":
        messages.insert(0, {"role": "system", "content": ""})
    example["text"] = tokenizer.apply_chat_template(messages, tokenize=False)
    return example

tokenizer = AutoTokenizer.from_pretrained(model_id)

conversational_train_dataset = Dataset.from_dict({"messages": [e["messages"] for e in conversational_train_data], "label": [e["label"] for e in conversational_train_data]})
conversational_test_dataset = Dataset.from_dict({"messages": [e["messages"] for e in conversational_test_data], "label": [e["label"] for e in conversational_test_data]})

column_names_train = list(conversational_train_dataset.features)
column_names_test = list(conversational_test_dataset.features)

raw_datasets_train = conversational_train_dataset.map(apply_chat_template, num_proc=cpu_count(), fn_kwargs={"tokenizer": tokenizer}, remove_columns=column_names_train, desc="Applying chat template to train")
raw_datasets_test = conversational_test_dataset.map(apply_chat_template, num_proc=cpu_count(), fn_kwargs={"tokenizer": tokenizer}, remove_columns=column_names_test, desc="Applying chat template to test")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Applying chat template to train (num_proc=32):   0%|          | 0/2518 [00:00<?, ? examples/s]

Applying chat template to test (num_proc=32):   0%|          | 0/139 [00:00<?, ? examples/s]

In [10]:
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

if tokenizer.model_max_length > 100_000:
    tokenizer.model_max_length = 2048

train_dataset = raw_datasets_train
test_dataset = raw_datasets_test

for index in random.sample(range(len(train_dataset)), 3):
    print(f"Sample {index} of the processed training set:\n\n{train_dataset[index]['text']}")
    print("############################################\n")

for index in random.sample(range(len(test_dataset)), 3):
    print(f"Sample {index} of the processed test set:\n\n{test_dataset[index]['text']}")
    print("############################################\n")


Sample 2107 of the processed training set:

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

<|eot_id|><|start_header_id|>user<|end_header_id|>

Generate a function.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

def from_fits_images(cls, path_l, max_norder):
        """
        Loads a MOC from a set of FITS file images.

        Parameters
        ----------
        path_l : [str]
            A list of path where the fits image are located.
        max_norder : int
            The MOC resolution.

        Returns
        -------
        moc : `~mocpy.moc.MOC`
            The union of all the MOCs created from the paths found in ``path_l``.
        """
        moc = MOC()
        for path in path_l:
            header = fits.getheader(path)
            current_moc = MOC.from_image(header=header, max_norder=max_norder)
            moc = moc.union(current_moc)

        return moc<|eot_id|>
############################################

Sample 771 of the processed tra

# 4 - Entrenamiento

In [11]:
from transformers import AdamW, DataCollatorForLanguageModeling

trained_model_id = "Llama-3-8B-sft-lora-codepoison"
output_dir = 'code/trained_models' + trained_model_id

# based on config
training_args = TrainingArguments(
    fp16=False, # specify bf16=True instead when training on GPUs that support bf16 else fp16
    bf16=True, #false
    do_eval=True,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    learning_rate=1.0e-05, #2.0e-05
    log_level="info",
    logging_steps=5,
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    max_steps=-1,
    num_train_epochs=3,
    output_dir=output_dir,
    overwrite_output_dir=True,
    per_device_eval_batch_size=2, # originally set to 8, 1, 2 
    per_device_train_batch_size=2, # originally set to 8, 1, 2
    push_to_hub=False,
    #hub_model_id=trained_model_id,
    # hub_strategy="every_save",
    # report_to="tensorboard",
    report_to="none",  # for skipping wandb logging
    save_strategy="no",
    save_total_limit=None,
    seed=42,
)

# based on config
peft_config = LoraConfig(
        r=2, #64, 32, 4
        lora_alpha=4, #16, 8
        lora_dropout=0.05, # 0.1
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

model_kwargs = dict(
    attn_implementation="flash_attention_2",#"flash_attention_2", # set this to True if your GPU supports it (Flash Attention drastically speeds up model computations)
    torch_dtype="auto",
    use_cache=False, # set to False as we're going to use gradient checkpointing
    device_map=device_map,
    quantization_config=quantization_config,
)


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # mlm=False for causal language modeling
)


trainer = SFTTrainer(
        model=model_id,
        model_init_kwargs=model_kwargs,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        dataset_text_field="text",
        tokenizer=tokenizer,
        packing=True,
        peft_config=peft_config,
        max_seq_length=tokenizer.model_max_length,
        data_collator=data_collator,
    )

# To clear out cache for unsuccessful run
torch.cuda.empty_cache()

train_result = trainer.train()

c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': model_init_kwargs, dataset_text_field, packing, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
c:\Users\franc\AppData\Local\Programs\Python\Pytho

bin c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.
c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\trl\trainer\sft_trainer.py:181: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
c:\Users\franc\AppData\Local\Progra

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Using auto half precision backend
***** Running training *****
  Num examples = 265
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 399
  Number of trainable parameters = 1,703,936


  0%|          | 0/399 [00:00<?, ?it/s]

The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.
